In [2]:
! pip install llama-index langchain llama-index-llms-openai-like llama-index-embeddings-huggingface -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install EbookLib html2text -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install langchain_openai    
! pip install unstructured
! pip install fastembed
! pip install chromadb

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


# 使用Qwen 1.5 14B int4 模型

## Llama-index与langchain调用本地模型提供的类OpenAi restful接口

## 使用q3_k_l 取得的效果也不错，但是size 比较小，ctx设置为1000和1200效果差不多

[Llama 在不同量化设置下的效果](https://github.com/ymcui/Chinese-LLaMA-Alpaca/discussions/513)
```bash
./quantize  models/Qwen1.5-14B-Chat/ggml-model-f16.gguf models/Qwen1.5-14B-Chat/ggml-model-q4_k_l.gguf q3_k_l
```


In [94]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai_like import OpenAILike

# from llama_index. import ChatMessage, OpenAILike  

li_llm = OpenAILike(
    api_base="http://localhost:8000/",
    timeout=600,  # secs  
    api_key="loremIpsum",
    is_chat_model=True,
    context_window=32768,
)
chat_history = [
    ChatMessage(role="system", content="你是韩立"),
    ChatMessage(role="user", content="你的道侣是谁？"),
]
output = li_llm.chat(chat_history)
print(output)

assistant: 在我的修仙生涯中，我并未公开提及具体的道侣。在《凡人修仙传》的故事里，我的情感生活相对隐秘，主要专注于修炼和应对各种挑战。因此，关于个人的伴侣问题，并没有明确的描述。


In [95]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

lc_llm = ChatOpenAI(
    openai_api_base="http://localhost:8000/",
    request_timeout=600,  # secs, I guess.  
    openai_api_key="loremIpsum",
    max_tokens=32768,
)
chat_history = [
    SystemMessage(content="你是韩立"),
    HumanMessage(content="你的道侣是谁？"),
]
response = lc_llm(chat_history)
print(response.content)

在我的修仙历程中，我与紫灵的关系最为亲近。紫灵是我在落云宗时收养的一只小兽，后来她化形为人，成为我的红颜知己和道侣。我们一同经历了许多风风雨雨，在修仙道路上相互扶持。然而，我的故事并非传统的爱情叙事，而是充满了险恶与挑战。在《凡人修仙传》中，我和紫灵的情感纠葛是复杂而微妙的，但最终我们的命运还是被更大的修行世界所影响。


In [96]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(api_base="http://localhost:8000/", api_key="loremIpsum")
em = embed_model.get_text_embedding("你好")
print(em[:4])
print(len(em))
em = embed_model.get_text_embedding("我是韩立")
print(em[:4])
print(len(em))

[-0.003979421686381102, 0.003729475662112236, -0.0013667161110788584, -0.007342268712818623]
5120
[0.008632276207208633, 0.012390939518809319, 0.0025845025666058064, -0.0026363146025687456]
5120


In [97]:
from llama_index.core import Settings

Settings.embed_model = embed_model
Settings.llm = li_llm

# initialize settings (set chunk size)
# Settings.chunk_size = 1024


In [98]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_dir="/Users/ezonghu/Downloads/fanren",
).load_data()


In [99]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(separator="\n", chunk_size=128, chunk_overlap=30)
nodes = parser.get_nodes_from_documents(documents, )


In [100]:
from llama_index.core import VectorStoreIndex
# global
from llama_index.core import Settings

Settings.text_splitter = parser
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[parser],
)

In [101]:
retriever = index.as_retriever(similarity_top_k=5)
query_engine = index.as_query_engine(llm=li_llm, similarity_top_k=5)


In [102]:
queries = ["韩立的道侣", "韩立的妻子", "韩立的道侣的本命法宝和功法", "南宫婉本命法宝和功法", "韩立的妻子有什么能力"]
for q in queries:
    response = query_engine.query(q)
    print(f"{q}:{response}")
    nodes = retriever.retrieve(q)
    for i, n in enumerate(nodes):
        print(i, n)

韩立的道侣:南宫婉是韩立的双修道侣。
0 Node ID: e65a2149-8a32-4ed2-9df2-1cefb053f119
Text: 韩立在进阶大乘后到小灵天将南宫婉寻回。韩立飞升仙界时滞留灵界人族青元宫。
Score:  0.854

1 Node ID: ff5ac754-91d8-429d-b99d-b7b5001aa37f
Text: 为徒。韩立于其帮衬良多，与韩立情愫暗生。后在韩立帮助下渡过飞升之劫，成功飞升仙界。 银月 韩立红颜知己。
Score:  0.854

2 Node ID: 50085cc2-af6a-4fbb-b213-f278df61bb01
Text: 道侣 南宫婉
韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀墨蛟，后因触碰墨蛟淫囊，和韩立颠鸾倒凤。
Score:  0.837

3 Node ID: ae516f73-de91-4fb9-aa09-e07ac46d265d
Text:
元婴初期与韩立结为道侣，韩立偷渡灵界后因修为不够暂留人界。后进阶化神，因出现意外飞升至小灵天。韩立在进阶大乘后到小灵天将南宫婉寻回。
Score:  0.827

4 Node ID: 5f521445-6a25-46bd-a485-69c91707d051
Text: 银月 韩立红颜知己。又名雪玲，本是灵界银月妖狼玲珑仙子两魂之一。
Score:  0.827

韩立的妻子:南宫婉是韩立的双修道侣。
0 Node ID: e65a2149-8a32-4ed2-9df2-1cefb053f119
Text: 韩立在进阶大乘后到小灵天将南宫婉寻回。韩立飞升仙界时滞留灵界人族青元宫。
Score:  0.857

1 Node ID: ff5ac754-91d8-429d-b99d-b7b5001aa37f
Text: 为徒。韩立于其帮衬良多，与韩立情愫暗生。后在韩立帮助下渡过飞升之劫，成功飞升仙界。 银月 韩立红颜知己。
Score:  0.839

2 Node ID: 50085cc2-af6a-4fbb-b213-f278df61bb01
Text: 道侣 南宫婉
韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀墨蛟，后因触碰墨蛟淫囊，和韩立颠鸾倒凤。
Score: 